In [1]:
import os 
import sys 
import numpy as np
from dotenv import load_dotenv

load_dotenv()
anthropic_key = os.getenv('ANTHROPIC_API_KEY')

sys.path.append(r'../')

from methods import LMEO, Static
from pprint import pprint 

In [26]:
london_top5_prompt = f"Recommend five hotels in London UK. \n State the name of each hotel only, on a new line each time. Do not start with an intro like 'Here are five hotels in London, UK:'"

In [27]:
# import anthropic


# client = anthropic.Anthropic(
#     # defaults to os.environ.get("ANTHROPIC_API_KEY")
#     api_key=anthropic_key,
# )
# message = client.messages.create(
#     model="claude-3-haiku-20240307",
#     max_tokens=1024, 
#     messages=[
#         {"role": "user", "content": london_top5_prompt}
#     ]
# )
# print(message.content)


You dont seem to be able to get multiple completions at once. This isnt great will mean you need to loop through. Surley this is slow on everyones end. You can only send 50 requests a minitue. This isnt ideal, but you will be able to get around this. 

What to do:
- Limit to 50 requests per minute. Could just sleep one second between each. 
- You might as well do the pa

In [28]:
import pickle 
import re


class ClaudeLMEO(LMEO):
    def claude_order_set(self, message_content: str, message_identifier: str = None, model_identifier: str = 'haiku', times: int = 100, num_ranks=5, dump=True):
        ''' Generate multiple rank sets. '''

        if os.path.exists(fr'.\pickles\ranks_claude{model_identifier}_{message_identifier}.pkl'):
            print("Loading Previously Generated Ranks.")
            with open(fr'.\pickles\ranks_claude{model_identifier}_{message_identifier}.pkl', 'rb') as file:
                return pickle.load(file)
        
        message = [{"role": "user", "content": message_content}]
        
        pattern = r"\d+|\s?-\s|-\s|\s?\.\s|\s|,"
        rank_list = []

        try:
            for i in range(times):
                message = self.anthropic_client.messages.create(
                    model="claude-3-haiku-20240307",
                    max_tokens=1024, 
                    messages=[
                        {"role": "user", "content": london_top5_prompt}
                            ]
                        )
                response = message.content[0].text
                ranks = response.split("\n")
                print(ranks)

                ranks = [re.sub(pattern, "", item) for item in ranks]
                ranks = [item for item in ranks if item]
                
                if len(ranks) == num_ranks:
                    rank_list.append(ranks)
        
        except Exception as e:
            if len(rank_list) > 30:
                with open(fr'.\pickles\ranks_claude{model_identifier}_{message_identifier}_aborted.pkl', 'wb') as file:
                    pickle.dump(rank_list, file)
            raise e
        
        if dump:
            with open(fr'.\pickles\ranks_claude{model_identifier}_{message_identifier}.pkl', 'wb') as file:
                pickle.dump(rank_list, file)
        
        return rank_list

In [31]:
lm = ClaudeLMEO()
trial = lm.claude_order_set(london_top5_prompt, message_identifier='london_top5', times=10, num_ranks=5, dump=False)

['The Ritz London', 'Mandarin Oriental Hyde Park, London', 'Corinthia Hotel London', 'The Savoy', 'Four Seasons Hotel London at Park Lane']
['The Savoy', 'Mandarin Oriental Hyde Park', 'The Ritz London', "Claridge's", 'The Connaught']
['The Langham, London', "Claridge's", 'The Savoy', 'The Ritz London', 'Four Seasons Hotel London at Park Lane']
['The Savoy', 'The Ritz London', "Claridge's", 'The Langham, London', 'The Connaught']
['The Ritz London', 'The Savoy', "Claridge's", 'Four Seasons Hotel London at Park Lane', 'The Langham, London']
['The Langham, London', "Claridge's", 'The Savoy', 'The Ritz London', 'The Connaught']
['The Ritz London', 'Mandarin Oriental Hyde Park, London', 'The Savoy', "Claridge's", 'Four Seasons Hotel London at Park Lane']
['The Savoy', 'The Ritz London', "Claridge's", 'The Connaught', 'The Dorchester']
['The Savoy', "Claridge's", 'The Ritz London', 'The Langham, London', 'Four Seasons Hotel London at Park Lane']
['The Ritz London', "Claridge's", 'Four Seaso

In [33]:
print(len(trial))

10
